# **Training Neural Network**
2023-01-30

1. Prerequisite
2. Activation
3. Optimizer
4. Regularization
5. FC vs Conv


# 1. Prerequisite

Mnist dataset에 대해서 DataLoader와 Trainer class를 생성



## Import packages

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils import data
print(torch.__version__)
print(torch.cuda.is_available())

1.13.1+cu116
True


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

## Load Dataset

In [3]:
mnist = fetch_openml('mnist_784', cache=False)
X = mnist.data.astype('float32').values
y = mnist.target.astype('int64').values
X /= 255.0
print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


## Split Dataset

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(56000, 784)
(56000,)
(14000, 784)
(14000,)


## Pytorch Dataset 

In [5]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        super(CustomDataset, self).__init__()
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        x = self.X[index]
        y = self.y[index]
        x = torch.from_numpy(x).float()
        y = torch.from_numpy(np.array(y)).long()
        return x, y

    def __len__(self):
        return len(self.X)

In [6]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

print(len(train_dataset))
print(train_dataset.X.shape)
print(len(test_dataset))
print(test_dataset.X.shape)

56000
(56000, 784)
14000
(14000, 784)


## DataLoader


In [7]:
batch_size = 64

# shuffle the train data
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# do not shuffle the val & test data
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# dataset size // batch_size
print(len(train_dataloader))
print(len(test_dataloader))

875
219


## Trainer


In [8]:
class Trainer():
    def __init__(self, trainloader, testloader, model, optimizer, criterion, device):
        """
        trainloader: train data's loader
        testloader: test data's loader
        model: model to train
        optimizer: optimizer to update your model
        criterion: loss function
        """
        self.trainloader = trainloader
        self.testloader = testloader
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device
        
    def train(self, epoch = 1):
        self.model.train()
        for e in range(epoch):
            running_loss = 0.0  
            for i, data in enumerate(self.trainloader, 0): 
                inputs, labels = data 
                # model에 input으로 tensor를 gpu-device로 보낸다
                inputs = inputs.to(self.device)  
                labels = labels.to(self.device)
                # zero the parameter gradients
                self.optimizer.zero_grad()    
                # forward + backward + optimize
                outputs = self.model(inputs) 
                loss = self.criterion(outputs, labels)  
                loss.backward() 
                self.optimizer.step() 
                running_loss += loss.item()
            
            print('epoch: %d  loss: %.3f' % (e + 1, running_loss / len(self.trainloader)))
            running_loss = 0.0
        
    def test(self):
        self.model.eval() 
        correct = 0
        for inputs, labels in self.testloader:
            inputs = inputs.to(self.device)
            labels = labels.to(self.device)
            output = self.model(inputs) 
            pred = output.max(1, keepdim=True)[1] # get the index of the max 
            correct += pred.eq(labels.view_as(pred)).sum().item()
        test_acc = correct / len(self.testloader.dataset)
        print('test_acc: %.3f' %(test_acc))

# 2. Activation Function

**sigmoid function** vs **relu function**

- input: 784
- hidden: 32 or (32, 32)
- output: 10
- **activation: sigmoid or relu**
- optimizer: sgd
- loss: cross-entropy

## 2-layer Network + Sigmoid

In [9]:
class MLP(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 hidden_dim=32, 
                 output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return x

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

In [10]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch: 1  loss: 2.161
epoch: 2  loss: 1.755
epoch: 3  loss: 1.326
epoch: 4  loss: 1.037
epoch: 5  loss: 0.855
epoch: 6  loss: 0.734
epoch: 7  loss: 0.649
epoch: 8  loss: 0.588
epoch: 9  loss: 0.541
epoch: 10  loss: 0.505


In [11]:
trainer.test()

test_acc: 0.882


## 2-layer Network + ReLU

In [12]:
class MLP(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 hidden_dim=32, 
                 output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

In [13]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 1.250
epoch: 2  loss: 0.515
epoch: 3  loss: 0.406
epoch: 4  loss: 0.364
epoch: 5  loss: 0.339
epoch: 6  loss: 0.322
epoch: 7  loss: 0.309
epoch: 8  loss: 0.298
epoch: 9  loss: 0.289
epoch: 10  loss: 0.281


In [14]:
trainer.test()

test_acc: 0.920


## 3-layer Network + Sigmoid

In [15]:
class MLP(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 hidden_dim=(32,32), 
                 output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim[0])
        self.fc2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.fc3 = nn.Linear(hidden_dim[1], output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        x = self.fc3(x)
        return x

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [16]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 2.303
epoch: 2  loss: 2.295
epoch: 3  loss: 2.289
epoch: 4  loss: 2.280
epoch: 5  loss: 2.265
epoch: 6  loss: 2.236
epoch: 7  loss: 2.182
epoch: 8  loss: 2.086
epoch: 9  loss: 1.949
epoch: 10  loss: 1.784


In [17]:
trainer.test()

test_acc: 0.522


## 3-layer Network + ReLU

In [18]:
class MLP(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 hidden_dim=(32,32), 
                 output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim[0])
        self.fc2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.fc3 = nn.Linear(hidden_dim[1], output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [19]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 1.857
epoch: 2  loss: 0.703
epoch: 3  loss: 0.453
epoch: 4  loss: 0.378
epoch: 5  loss: 0.342
epoch: 6  loss: 0.318
epoch: 7  loss: 0.298
epoch: 8  loss: 0.281
epoch: 9  loss: 0.266
epoch: 10  loss: 0.253


In [20]:
trainer.test()

test_acc: 0.927


#### **Activation Function과 layer 수에 따른 성능의 차이**



1. sigmoid

`Sigmoid` 함수는 binary classification 에 적절함 함수다. 일정 값을 기준으로 `0`과 `1`로 분류하는 방식이다. Deep Learning에서는 특정 임계치를 넘을 때만 활성화되기 때문에 activation function 중의 하나로 구분되는 함수다.

보통 처음은 input layer, 마지막은 output layer 라고 하는데, 이 가운데 보이지 않는 부분은 hidden layer로 칭한다. 실제로 9개의 hidden layer 가 있다고 했을 때 Tensorflow에 돌려 보면 정확도가 0.5밖에 안되는.. 1개 hidden layer일때 보다도 못한 결과가 나오게 된다.

이는 `backpropagation`에서의 `Vanishing Gradient` 때문에 발생한다. **2-layer 또는 3-layer 정도는 학습이 잘 되지만 더 깊은 층에서 학습이 되지 않는** 이유는 layer가 많을 경우 각각의 단계의 값을 미분해서 최초 레이어까지 결과 값을 전달해가게 되는데, 만약 내부의 hidden layer들이 모두 `sigmoid` 함수로 이루어져 있다면 각 단계에서 계산한 값은 모두 `0`과 `1` 사이의 값일 수밖에 없다. 따라서 여러 레이어를 갖고 있을 때, 최초 입력 값은 각각의 레이어에서 나온 값들을 곱해준 만큼의 결과에 영향을 주는 것이므로 최종 미분값은 결국 0에 가까운 값이 될 수 밖에 없다. 이를 경사도(기울기)가 사라지는 현상으로 본다. 최초 입력 값이 최종 결과 값에 별로 영향을 끼치지 않는다는 결론으로 수렴하게 되는 것이다.
sigmoid 함수는 $0< n <1$ 사이의 값만 다루므로 결국 `chain rule`을 이용해 계속 값을 곱해나간다고 했을 때 결과 값이 `0`에 수렴할 수 밖에 없다는 한계를 가지고 있으므로, 나중에는 `1`보다 작아지지 않게 하기 위한 대안으로 `ReLU`라는 함수를 적용하게 된다.


\\


2. ReLU

이후 내부 hidden layer를 활성화 시키는 함수로 `sigmoid`를 사용하지 않고 `ReLU`라는 활성화 함수를 사용하게 되는데, 이 함수는 쉽게 말해 0보다 작은 값이 나온 경우 0을 반환하고, 0보다 큰 값이 나온 경우 그 값을 그대로 반환하는 함수다. 0보다 큰 값일 경우 1을 반환하는 `sigmoid`와 다르다. 따라서 내부 hidden layer에는 `ReLU`를 적용하고, 마지막 output layer에서만 sigmoid 함수를 적용하면 이전에 비해 정확도가 훨씬 올라가게 된다.

\\

-> 결론적으로 활성화 함수는 입력값을 non-linear한 방식으로 출력값을 도출하기 위해 사용한다. 이를 통해 linear system을 non-linear한 system으로 바꿀 수 있게 되는 것이다.


# 3. Optimization

**SGD** vs **Momentum** vs **Adam ...**

- input: 784
- hidden: (32, 32)
- output: 10
- activation: relu
- **optimizer: sgd or momentum or adam**
- loss: cross-entropy

In [21]:
class MLP(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 hidden_dim=(32,32), 
                 output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim[0])
        self.fc2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.fc3 = nn.Linear(hidden_dim[1], output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

## 3-layer Network + ReLU + SGD

In [22]:
model = MLP()
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [23]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 1.901
epoch: 2  loss: 0.716
epoch: 3  loss: 0.456
epoch: 4  loss: 0.384
epoch: 5  loss: 0.348
epoch: 6  loss: 0.323
epoch: 7  loss: 0.303
epoch: 8  loss: 0.288
epoch: 9  loss: 0.275
epoch: 10  loss: 0.264


In [24]:
trainer.test()

test_acc: 0.922


## 3-layer Network + ReLU + Momentum





In [25]:
model = MLP()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.99)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [26]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 0.596
epoch: 2  loss: 0.276
epoch: 3  loss: 0.222
epoch: 4  loss: 0.196
epoch: 5  loss: 0.201
epoch: 6  loss: 0.166
epoch: 7  loss: 0.159
epoch: 8  loss: 0.177
epoch: 9  loss: 0.151
epoch: 10  loss: 0.137


In [27]:
trainer.test()

test_acc: 0.947


## 3-layer Network + ReLU + Adam



In [28]:
model = MLP()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [29]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 0.296
epoch: 2  loss: 0.171
epoch: 3  loss: 0.147
epoch: 4  loss: 0.142
epoch: 5  loss: 0.126
epoch: 6  loss: 0.125
epoch: 7  loss: 0.119
epoch: 8  loss: 0.114
epoch: 9  loss: 0.112
epoch: 10  loss: 0.104


In [30]:
trainer.test()

test_acc: 0.954


### Stochastic Gradient Descent (SGD)
SGD는 현재 위치에서 기울어진 방향이 전체적인 최솟값과 다른 방향을 가리키므로 지그재그 모양으로 탐색해나간다. 즉, SGD의 단점은 비등방성(anisotropy)함수에서는 탐색 경로가 비효율적이라는 것이다. 무작정 기울어진 방향으로 나아가는 방식보다 더 효율적인 방식이 필요하다. SGD를 보완한 기법으로 Momentum, AdaGrad, Adam이 있다.
```python
class SGD:
    def __init__(self, lr = 0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]
```

### Momentum

SGD와 달리 새로운 변수 v가 나오는데 이는 물리에서 말하는 속도(velocity)이다. Momentum은 '운동량'을 뜻하는 단어로 기울기 방향으로 힘을 받아 물체가 가속되어 공이 구르는 듯한 움직임을 보인다. 위의 식에서 알파가 가속도와 관련된 파라미터이다. SGD와 최적화 과정을 비교하면 지그재그 정도가 덜하다. x축의 힘은 작지만 방향이 변하지 않으므로 일정하게 가속하여 SGD보다 x축 방향으로 빠르게 나아가므로 지그재그 정도가 덜한 것이다.

```python
class Momentum:
    def __init__(self, ir = 0.01, momentum = 0.9):
        self.lr = lr
        self.momentum = momemtum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
            
            for key in parmas.keys():
                self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
                params[key] += self.v[key]
```

### Adam

공 구르는 듯한 Momentum과 적응적인 AdaGrad를 융합한 기법이 Adam이다. Adam은 학습률, 일차 모멘텀용 계수, 이차 모멘텀용 계수로 3개의 hyperparameter를 설정한다. 일차 모멘텀용 계수=0.9, 이차 모멘텀용 계수=0.999를 기본 설정값으로 하면 보통 좋은 결과를 얻을 수 있다.

\\

- **AdaGrad**

신경망 학습에서 학습률(learning rate)을 적절하게 설정하는 것이 중요하다. 학습률을 적절하게 설정하기 위해 학습률 감소(learning rate decay)라는 기술을 사용하는데 이는 학습 진행 중에 learning rate을 줄여가는 방법이다. 즉 처음에는 크게 학습을 하다가 점점 작게 학습한다는 의미이다. 이 방법을 발전시킨 것이 AdaGrad이며 AdaGrad는 각 매개변수에 Adaptive하게 조정하여 맞춤형 learning rate을 만든다.

SGD와 달리 새로운 변수 h가 나오는데 이는 기존의 기울기 값을 제곱한 값을 더하여 학습률을 조정하기 위한 변수이다(⊙기호는 행렬의 원소별 곱셈을 의미함). 매개변수의 원소 중 크게 갱신된 원소는 학습률이 낮아지며 학습률 감소가 매개변수의 원소마다 다르게 적용된다.

```python
class AdaGrad:
    def __init__(self, lr = 0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)
```

## 4. Regularization

image data batch-normalization

- input: 784
- hidden: 32 or (32, 32)
- output: 10
- activation: relu
- optimizer: adam
- **regularizer: batch_norm**
- loss: cross-entropy

## 3-layer Network + ReLU + Adam + batch_norm

In [31]:
class MLP(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 hidden_dim=(32,32), 
                 output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim[0])
        self.bn1 = nn.BatchNorm1d(hidden_dim[0])
        self.fc2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.bn2 = nn.BatchNorm1d(hidden_dim[1])
        self.fc3 = nn.Linear(hidden_dim[1], output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [32]:
model = MLP()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [33]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 0.257
epoch: 2  loss: 0.141
epoch: 3  loss: 0.116
epoch: 4  loss: 0.101
epoch: 5  loss: 0.091
epoch: 6  loss: 0.086
epoch: 7  loss: 0.079
epoch: 8  loss: 0.075
epoch: 9  loss: 0.071
epoch: 10  loss: 0.068


In [34]:
trainer.test()

test_acc: 0.971


In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

26634

#### Batch Normalization

``` markdown
각 레이어의 입력의 분산을 평균 0, 표준편차 1인 입력값으로 정규화 시키는 방법을 Whitening 이라고 하는데 들어오는 입력값의 특징들을 uncorrelated 하게
만들어주고, 각각의 분산을 1로 만들어주는 작업이다. 이는 covariance matrix의 계산과 inverse의 계산이 필요하기 때문에 계산량이 많을 뿐더러, Whitening은 
일부 파라미터들의 영향이 무시된다. 예를 들어 입력 값 X를 받아 Z = WX + b라는 결과를 내놓고 적절한 bias b 를 학습하려는 네트워크에서 Z에 E(Z) 를 
빼주는 작업을 한다고 생각해보면, 이 과정에서 b 값이 결국 빠지게 되고, 결과적으로 b의 영향은 없어지게 된다.
```

Whitening의 문제점을 해결하도록 한 트릭이 **Batch Normalization**이다. 배치 정규화는 평균과 분산을 조정하는 과정이 별도의 과정으로 떼어진 것이 아니라, 신경망 안에 포함되어 학습 시 평균과 분산을 조정하는 과정 역시 같이 조절된다는 점이 단순 Whitening 과는 구별된다. 즉, 각 레이어마다 정규화 하는 레이어를 두어, 변형된 분포가 나오지 않도록 조절하게 하는 것이 배치 정규화이다. 

배치 정규화는 간단히 말하자면 미니배치의 평균과 분산을 이용해서 정규화 한 뒤에, scale 및 shift 를 감마(γ) 값, 베타(β) 값을 통해 실행한다. 이 때 감마와 베타 값은 학습 가능한 변수이다. 즉, Backpropagation을 통해서 학습이 된다.

정규화 된 값을 활성화 함수의 입력으로 사용하고, 최종 출력 값을 다음 레이어의 입력으로 사용하는 것이다.

기존 $output = g(Z),\ Z = WX + b$ 식은 $output = g(BN(Z)),\ Z = WX + b$ 로 변경되는 것이다. 

# 5. Fully-Connected Layer vs Convolution Layer

모든 layer를 fully-connected layer로 만드는 것은 엄청난 파라미터와 연산량을 필요로 하기 때문에 더욱 큰 고화질의 이미지 데이터를 처리하는데는 적합하지 않다. 

## Convolution Operation

In [52]:
class Conv(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 output_dim=10):
        super(Conv, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels=8,
                               kernel_size=7,
                               stride=2)
        self.conv2 = nn.Conv2d(in_channels=8,
                               out_channels=8,
                               kernel_size=7,
                               stride=2)
        self.fc = nn.Linear(3*3*8, output_dim)

    def forward(self, x):
        # should reshape data into image
        x = x.reshape(-1, 1, 28, 28)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = x.reshape(-1, 3*3*8)
        x = self.fc(x)
        return x

In [53]:
model = Conv()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model.to(device)

Conv(
  (conv1): Conv2d(1, 8, kernel_size=(7, 7), stride=(2, 2))
  (conv2): Conv2d(8, 8, kernel_size=(7, 7), stride=(2, 2))
  (fc): Linear(in_features=72, out_features=10, bias=True)
)

In [54]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 0.190
epoch: 2  loss: 0.090
epoch: 3  loss: 0.084
epoch: 4  loss: 0.076
epoch: 5  loss: 0.076
epoch: 6  loss: 0.075
epoch: 7  loss: 0.072
epoch: 8  loss: 0.069
epoch: 9  loss: 0.076
epoch: 10  loss: 0.067


In [55]:
trainer.test()

test_acc: 0.973


In [56]:
count_parameters(model)

4274

고전적인 신경망 구조는 컴퓨터 영상 인식 작업에 비효율적인 것으로 밝혀졌다. 이미지는 신경 네트워크에 대한 대용량 이미지의 입력(수백 또는 수천 픽셀과 최대 3가지 색 채널을 가짐)을 하게 되므로 이 입력에 대한 처리 속도를 못 따라간다. 하지만 컴퓨팅 시스템의 발전으로 이와 같은 문제가 해결되었다.

​FCN(Fully Connected Networks)에서는 엄청난 수의 연결과 네트워크 매개변수를 필요로 한다. CNN Architectures은 입력 이미지가 각각의 객체나 사물의 특징으로 구성되었다는 데이터를 활용하고, 각 객체 및 형상들을 분리하여 해석하는 메커니즘을 만들어 전체적인 이미지가 무엇인지 알려준다.

또한, CNN 네트워크의 일부로서, convolution/pooling 네트워크 프로세스의 최종 결과를 취하여 분류 결정에 도달하는 완전히 연결된 계층(Fully Connected Layer)이 있다.

Fully connected layer의 목적은 Convolution/Pooling 프로세스의 결과를 취하여 이미지를 정의된 라벨로 분류하는 데 사용한다.

Convolution/Pooling 의 출력은 각각 특정 입력 이미지내의 객체가 라벨에 속할 확률을 나타내는 값의 단일 벡터로 평탄화 된다. 예를 들어, 이미지가 고양이인 경우, 수염이나 털과 같은 것을 나타내는 형상은 "cat" 라벨에 대한 높은 확률을 가져야 한다.

미리 학습된 Weight에 의해 계산되어지고 인공신경망에서처럼 활성화 함수(일반적으로 ReLu)를 통과하고 출력층으로 전달되는데, 그 안에서 모든 뉴런은 이미 정의된 분류 라벨을 나타낸다.​

FC(Fully connected layer)를 정의하자면, 완전히 연결 되었다라는 뜻으로, 한 층의 모든 뉴런이 다음층이 모든 뉴런과 연결된 상태로 2차원의 배열 형태 이미지를 1차원의 평탄화 작업을 통해 이미지를 분류하는데 사용되는 계층이다.


1. 2차원 배열 형태의 이미지를 1차원 배열로 평탄화

2. 활성화 함수(Relu, Leaky Relu, Tanh,등)뉴런을 활성화

3. 분류기(Softmax) 함수로 분류



완전히 연결된 계층에 대한 컨볼루션 계층의 강점은 정확히 그것들이 완전히 연결된 계층보다 더 좁은 범위의 특징을 나타낸다는 것이다. 완전히 연결된 층의 뉴런은 앞의 층의 모든 뉴런에 연결되어 있으므로 앞의 층의 뉴런 중 하나가 변경되면 변경될 수 있다. 그러나 컨볼루션 레이어의 뉴런은 컨볼루션 커널의 너비 내에서 이전 레이어의 "근처" 뉴런에만 연결된다. 결과적으로, 컨볼루션 레이어의 뉴런들은 이전 레이어의 뉴런들 대부분의 활성화에 민감하지 않다는 점에서 더 좁은 범위의 특징들을 나타낼 수 있다.

이러한 방식으로 기능 범위를 제한하면 대부분의 정보가 로컬일 것으로 예상되는 경우에 유용할 수 있습니다. 예를 들어, 이미지 분류에서 새는 이미지의 위치와 이미지의 다른 곳에 자동차가 있는지 여부에 관계없이 새의 위치에 있는 픽셀을 기준으로 새처럼 보입니다. 이 사전 예상의 유용성은 완전히 무작위 가중치를 가진 CNN조차도 완전히 훈련된 CNN만큼 분류에 유용한 기능을 제공한다는 관찰에 의해 도출된다.

### under 20000 Parameter

$$
Output\ height = (Input\ height + padding\ height\ top + padding\ height\ bottom - kernel\ height) / (stride\ height) + 1\\\ \\ 
Output\ width = (Output\ width + padding\ width\ right + padding\ width\ left - kernel\ width) / (stride\ width) + 1
$$

In [57]:
class CustomModel(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 output_dim=10):
        super(CustomModel, self).__init__()
        #28*28*1
        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels=8,
                               kernel_size=(7, 7),
                               stride=2)
        #14*14*8
        self.conv2 = nn.Conv2d(in_channels=8,
                               out_channels=16,
                               kernel_size=(7,7),
                               stride=1)
        #7*7*16
        self.pool1 = nn.MaxPool2d(2, stride=2)
        self.fc = nn.Linear(8*8, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        # should reshape data into image
        x = x.reshape(-1, 1, 28, 28)
        x = self.conv1(x)
        x = F.relu(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = F.softmax(x)
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.fc2(x)
        return x

    

In [58]:
model = CustomModel()
print(count_parameters(model))
if count_parameters(model) > 20000:
  raise AssertionError

9098


In [59]:
optimizer = optim.Adam(model.parameters(), lr=0.0003)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model.to(device)

CustomModel(
  (conv1): Conv2d(1, 8, kernel_size=(7, 7), stride=(2, 2))
  (conv2): Conv2d(8, 16, kernel_size=(7, 7), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

In [62]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 25)

<ipython-input-57-173cb3bcb9c4>:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


epoch: 1  loss: 0.046
epoch: 2  loss: 0.044
epoch: 3  loss: 0.043
epoch: 4  loss: 0.041
epoch: 5  loss: 0.039
epoch: 6  loss: 0.038
epoch: 7  loss: 0.037
epoch: 8  loss: 0.035
epoch: 9  loss: 0.034
epoch: 10  loss: 0.033
epoch: 11  loss: 0.032
epoch: 12  loss: 0.031
epoch: 13  loss: 0.029
epoch: 14  loss: 0.029
epoch: 15  loss: 0.028
epoch: 16  loss: 0.027
epoch: 17  loss: 0.026
epoch: 18  loss: 0.025
epoch: 19  loss: 0.025
epoch: 20  loss: 0.024
epoch: 21  loss: 0.023
epoch: 22  loss: 0.022
epoch: 23  loss: 0.021
epoch: 24  loss: 0.021
epoch: 25  loss: 0.020


In [63]:
trainer.test()

<ipython-input-57-173cb3bcb9c4>:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


test_acc: 0.982
